Caim中，每個險種展開，  
以policy_number先找出所有的claim_number  
每個claim_number每個險種的 已決賠償, 已決費用的sum, mean, std1跟 未賠償比例，次數是依據該次claim_number中該險種的列數
每個claim_number從各險種的 已決賠償, 已決費用的sum, mean, std1跟 未賠償比例，次數是依據claim_number數
Accidents_counts出事次數	At_Fault_mean肇責比例平均

In [1]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import math
import numpy as np
import datetime

import time
from IPython.display import clear_output

%matplotlib inline
plt.rcParams['font.family'] = ['SimHei'] #  set font for chinese in matplotlib figure

In [2]:
df_policy_raw = pd.read_csv('../Tbrain_Insurance/policy_claim/policy_0702.csv')
df_claim_raw = pd.read_csv('../Tbrain_Insurance/policy_claim/claim_0702.csv')
df_training_raw = pd.read_csv('../Tbrain_Insurance/training-set.csv')
df_testing_raw = pd.read_csv('../Tbrain_Insurance/testing-set.csv')

In [3]:
df_policy = df_policy_raw 
df_claim = df_claim_raw 
df_training = df_training_raw
df_testing = df_testing_raw
pd.set_option('display.max_columns', None)

In [125]:
df_claim['Coverage'].nunique()

48

In [353]:
df_claim_test_policy_number = df_training.loc[df_training['Next_Premium']==0]['Policy_Number'].head(10)

In [186]:
df_policy.loc[df_policy['Policy_Number']=='79edd5355bf511649ffefe614bbd6e46d4a00474']

,Policy_Number,Insured's_ID,Prior_Policy_Number,Cancellation,Vehicle_identifier,Vehicle_Make_and_Model1,Vehicle_Make_and_Model2,Manafactured_Year_and_Month,Engine_Displacement_(Cubic_Centimeter),Imported_or_Domestic_Car,Coding_of_Vehicle_Branding_&_Type,qpt,fpt,Main_Insurance_Coverage_Group,Insurance_Coverage,Insured_Amount1,Insured_Amount2,Insured_Amount3,Coverage_Deductible_if_applied,Premium,Replacement_cost_of_insured_vehicle,Distribution_Channel,Multiple_Products_with_TmNewa_(Yes_or_No?),lia_class,plia_acc,pdmg_acc,fassured,ibirth,fsex,fmarriage,aassured_zip,iply_area,dbirth,fequipment1,fequipment2,fequipment3,fequipment4,fequipment5,fequipment6,fequipment9,nequipment9
1137503,79edd5355bf511649ffefe614bbd6e46d4a00474,Qd7194ceb3b068ae4702b49817eee5b396508cdae,NaN,Y,fd0a1696e8332e414f8fc47ef769e66d30489663,dda1b15cf567d2402a0d1249ee12e9e072bb3120,fe81499d07934b745eb02c8f6f345edfcc1a1756,2011,2400,10,894c70715243a268e058e2217eb16de54a84eb3c,5.0,P,車損,04M,0,0,433000,0,12501,113.9,39005d92752d56310b3a357f42b57f304621689a,0.0,6,0.3,0.2,1,03/1983,1,1,b202977c0fc07e1c6b31961a905395a8ffb23121,761f22b2c1593d0bb87e0b606f990ba4974706de,03/1983,0,0,0,0,0,0,0,
1137504,79edd5355bf511649ffefe614bbd6e46d4a00474,Qd7194ceb3b068ae4702b49817eee5b396508cdae,NaN,Y,fd0a1696e8332e414f8fc47ef769e66d30489663,dda1b15cf567d2402a0d1249ee12e9e072bb3120,fe81499d07934b745eb02c8f6f345edfcc1a1756,2011,2400,10,894c70715243a268e058e2217eb16de54a84eb3c,5.0,P,車損,05E,0,0,433000,0,188,113.9,39005d92752d56310b3a357f42b57f304621689a,0.0,6,0.3,0.2,1,03/1983,1,1,b202977c0fc07e1c6b31961a905395a8ffb23121,761f22b2c1593d0bb87e0b606f990ba4974706de,03/1983,0,0,0,0,0,0,0,
1137505,79edd5355bf511649ffefe614bbd6e46d4a00474,Qd7194ceb3b068ae4702b49817eee5b396508cdae,NaN,Y,fd0a1696e8332e414f8fc47ef769e66d30489663,dda1b15cf567d2402a0d1249ee12e9e072bb3120,fe81499d07934b745eb02c8f6f345edfcc1a1756,2011,2400,10,894c70715243a268e058e2217eb16de54a84eb3c,5.0,P,竊盜,05N,0,0,433000,10,864,113.9,39005d92752d56310b3a357f42b57f304621689a,0.0,6,0.3,0.2,1,03/1983,1,1,b202977c0fc07e1c6b31961a905395a8ffb23121,761f22b2c1593d0bb87e0b606f990ba4974706de,03/1983,0,0,0,0,0,0,0,
1137506,79edd5355bf511649ffefe614bbd6e46d4a00474,Qd7194ceb3b068ae4702b49817eee5b396508cdae,NaN,Y,fd0a1696e8332e414f8fc47ef769e66d30489663,dda1b15cf567d2402a0d1249ee12e9e072bb3120,fe81499d07934b745eb02c8f6f345edfcc1a1756,2011,2400,10,894c70715243a268e058e2217eb16de54a84eb3c,5.0,P,竊盜,09@,0,0,433000,0,130,113.9,39005d92752d56310b3a357f42b57f304621689a,0.0,6,0.3,0.2,1,03/1983,1,1,b202977c0fc07e1c6b31961a905395a8ffb23121,761f22b2c1593d0bb87e0b606f990ba4974706de,03/1983,0,0,0,0,0,0,0,
1137507,79edd5355bf511649ffefe614bbd6e46d4a00474,Qd7194ceb3b068ae4702b49817eee5b396508cdae,NaN,Y,fd0a1696e8332e414f8fc47ef769e66d30489663,dda1b15cf567d2402a0d1249ee12e9e072bb3120,fe81499d07934b745eb02c8f6f345edfcc1a1756,2011,2400,10,894c70715243a268e058e2217eb16de54a84eb3c,5.0,P,竊盜,10A,0,0,30000,0,164,113.9,39005d92752d56310b3a357f42b57f304621689a,0.0,6,0.3,0.2,1,03/1983,1,1,b202977c0fc07e1c6b31961a905395a8ffb23121,761f22b2c1593d0bb87e0b606f990ba4974706de,03/1983,0,0,0,0,0,0,0,
1137508,79edd5355bf511649ffefe614bbd6e46d4a00474,Qd7194ceb3b068ae4702b49817eee5b396508cdae,NaN,Y,fd0a1696e8332e414f8fc47ef769e66d30489663,dda1b15cf567d2402a0d1249ee12e9e072bb3120,fe81499d07934b745eb02c8f6f345edfcc1a1756,2011,2400,10,894c70715243a268e058e2217eb16de54a84eb3c,5.0,P,車責,15F,0,0,10000000,100,1040,113.9,39005d92752d56310b3a357f42b57f304621689a,0.0,6,0.3,0.2,1,03/1983,1,1,b202977c0fc07e1c6b31961a905395a8ffb23121,761f22b2c1593d0bb87e0b606f990ba4974706de,03/1983,0,0,0,0,0,0,0,
1137509,79edd5355bf511649ffefe614bbd6e46d4a00474,Qd7194ceb3b068ae4702b49817eee5b396508cdae,NaN,Y,fd0a1696e8332e414f8fc47ef769e66d30489663,dda1b15cf567d2402a0d1249ee12e9e072bb3120,fe81499d07934b745eb02c8f6f345edfcc1a1756,2011,2400,10,894c70715243a268e058e2217eb16de54a84eb3c,5.0,P,車責,16G,3000000,0,6000000,0,2683,113.9,39005d92752d56310b3a357f42b57f304621689a,0.0,6,0.3,0.2,1,03/1983,1,1,b202977c0fc07e1c6b3

In [343]:
df_policy.loc[df_policy['Policy_Number']=='2fe6a2d91ae7edcc7c63cb9701262d1421dbfc86']['Premium'].sum()

13596

In [342]:
df_training.loc[df_training['Policy_Number']=='2fe6a2d91ae7edcc7c63cb9701262d1421dbfc86']

,Policy_Number,Next_Premium
58482,2fe6a2d91ae7edcc7c63cb9701262d1421dbfc86,25623


In [344]:
df_claim.loc[(df_claim['Policy_Number']=='2fe6a2d91ae7edcc7c63cb9701262d1421dbfc86')][['Claim_Number', 'Nature_of_the_claim','At_Fault?','Accident_Date','Paid_Loss_Amount','paid_Expenses_Amount','Salvage_or_Subrogation?','Coverage']]

,Claim_Number,Nature_of_the_claim,At_Fault?,Accident_Date,Paid_Loss_Amount,paid_Expenses_Amount,Salvage_or_Subrogation?,Coverage
16252,08983057eeec099f6b37ff17d162bd248e894f89,1,100.0,2016/06,0,0,0,04M
16253,08983057eeec099f6b37ff17d162bd248e894f89,1,100.0,2016/06,0,0,0,05E
16254,08983057eeec099f6b37ff17d162bd248e894f89,1,100.0,2016/06,9390,0,0,16P
17610,346835c76557252f1b90826667230e341e884bc3,1,30.0,2016/07,0,0,0,04M
17611,346835c76557252f1b90826667230e341e884bc3,2,30.0,2016/07,0,0,0,04M
17612,346835c76557252f1b90826667230e341e884bc3,1,30.0,2016/07,18714,0,0,05E
17613,346835c76557252f1b90826667230e341e884bc3,2,30.0,2016/07,0,0,11000,05E
17614,346835c76557252f1b90826667230e341e884bc3,1,30.0,2016/07,0,0,0,16P
17615,346835c76557252f1b90826667230e341e884bc3,2,30.0,2016/07,0,0,0,16P
18190,72f764116423deb19f5f5e4485cd411938b0b50c,1,100.0,2016/06,0,0,0,04M


In [358]:
df_claim_test = df_claim.loc[df_claim['Policy_Number'].isin(['2fe6a2d91ae7edcc7c63cb9701262d1421dbfc86', '97f367ab2735fd7bcab7067398aa0e54b8ce1a33'])][['Claim_Number','Policy_Number', 'Nature_of_the_claim','At_Fault?','Accident_Date','Paid_Loss_Amount','paid_Expenses_Amount','Salvage_or_Subrogation?','Coverage']]

In [359]:
df_claim_test

,Claim_Number,Policy_Number,Nature_of_the_claim,At_Fault?,Accident_Date,Paid_Loss_Amount,paid_Expenses_Amount,Salvage_or_Subrogation?,Coverage
6126,65be5030fd24ee28677a677a13d62aaa021b28d2,97f367ab2735fd7bcab7067398aa0e54b8ce1a33,1,30.0,2015/10,0,0,0,40M
6444,043ef51fcd744fa9f8d45b259b1cb84c27dc15ed,97f367ab2735fd7bcab7067398aa0e54b8ce1a33,1,0.0,2015/10,77840,0,0,03L
6445,043ef51fcd744fa9f8d45b259b1cb84c27dc15ed,97f367ab2735fd7bcab7067398aa0e54b8ce1a33,2,0.0,2015/10,0,0,0,03L
6446,043ef51fcd744fa9f8d45b259b1cb84c27dc15ed,97f367ab2735fd7bcab7067398aa0e54b8ce1a33,2,0.0,2015/10,0,0,0,03L
6447,043ef51fcd744fa9f8d45b259b1cb84c27dc15ed,97f367ab2735fd7bcab7067398aa0e54b8ce1a33,2,0.0,2015/10,0,0,0,03L
6448,043ef51fcd744fa9f8d45b259b1cb84c27dc15ed,97f367ab2735fd7bcab7067398aa0e54b8ce1a33,1,0.0,2015/10,478160,700,0,04M
6449,043ef51fcd744fa9f8d45b259b1cb84c27dc15ed,97f367ab2735fd7bcab7067398aa0e54b8ce1a33,2,0.0,2015/10,0,0,45000,04M
6450,043ef51fcd744fa9f8d45b259b1cb84c27dc15ed,97f367ab2735fd7bcab7067398aa0e54b8ce1a33,2,0.0,2015/10,0,0,260000,04M
6451,043ef51fcd744fa9f8d45b259b1cb84c27dc15ed,97f367ab2735fd7bcab7067398aa0e54b8ce1a33,2,0.0,2015/10,0,7060,100000,04M
6452,043ef51fcd744fa9f8d45b259b1cb84c27dc15ed,97f367ab2735fd7bcab7067398aa0e54b8ce1a33,1,0.0,2015/10,34800,0,0,07P


In [363]:
df_claim_coverage=pd.DataFrame()
df_claim_policy_number_unique = df_claim['Policy_Number'].unique()
all_number = len(df_claim_policy_number_unique)
print(all_number)

appended_df_claim_coverage=[]
j=1
t0 = time.time()
for policy_number in df_claim_policy_number_unique:
    if j % 1000 == 0:
        t1 = time.time()
        print("Processing file {} ({}%)".format(j, 100*j//all_number), end="")
    
    df_temp = df_claim.loc[(df_claim['Policy_Number'] == policy_number) & (df_claim['Nature_of_the_claim'] == 1)]
    #print(df_temp)
    df_claim_coverage_each=pd.DataFrame()
    df_claim_coverage_each.at[0,'Policy_Number'] = policy_number
    accidents_counts = df_temp['Claim_Number'].nunique()
           
    coverage_unique = df_temp['Coverage'].unique()
    for coverage in coverage_unique:
        df_temp2 = df_temp.loc[(df_temp['Coverage'] == coverage)]
        #print(df_temp2)
        #print((df_temp2['Paid_Loss_Amount']==0).sum() )
        df_claim_coverage_each.at[0,coverage + '_noPaid_percent'] = (df_temp2['Paid_Loss_Amount']==0).sum() / len(df_temp2.index)
        df_claim_coverage_each.at[0,coverage + '_Paid_sum'] = df_temp2['Paid_Loss_Amount'].sum()
        df_claim_coverage_each.at[0,coverage + '_Paid_mean'] = df_temp2['Paid_Loss_Amount'].mean()
        df_claim_coverage_each.at[0,coverage + '_Paid_std'] = df_temp2['Paid_Loss_Amount'].std()
        
    df_claim_coverage_each.at[0,'Accidents_counts'] = accidents_counts
    df_claim_coverage_each.at[0,'Paid_sum'] = df_temp['Paid_Loss_Amount'].sum()
    paid_mean = df_temp['Paid_Loss_Amount'].sum()/accidents_counts
    df_claim_coverage_each.at[0,'Paid_mean'] = paid_mean
    
    df_claim_coverage_each.at[0,'paid_Expenses_Amount_sum'] = df_temp['paid_Expenses_Amount'].sum()
    paid_Expenses_Amount_mean = df_temp['paid_Expenses_Amount'].sum()/accidents_counts
    df_claim_coverage_each.at[0,'paid_Expenses_Amount_mean'] = paid_Expenses_Amount_mean
    
    paid_amount_per_claim = []
    paid_Expenses_per_claim = []
    At_Fault_per_claim = []
    claim_unique = df_temp['Claim_Number'].unique()
    for claim_number in claim_unique:
        df_temp3 = df_temp.loc[(df_temp['Claim_Number'] == claim_number)]
        #print(df_temp3)
        paid_amount_per_claim.append(df_temp3['Paid_Loss_Amount'].sum())
        paid_Expenses_per_claim.append(df_temp3['paid_Expenses_Amount'].sum())
        At_Fault_per_claim.append(df_temp3['At_Fault?'].mean())
                
    #print(At_Fault_per_claim)
    #print(paid_amount_per_claim)
    #print(paid_Expenses_per_claim)
    df_claim_coverage_each.at[0,'noPaid_percent'] = paid_amount_per_claim.count(0) / accidents_counts
    
    fault = 0
    for i in At_Fault_per_claim:
        fault += i
    df_claim_coverage_each.at[0,'At_Fault_mean'] = fault / accidents_counts
    
    square_sum = 0
    for amount in (paid_amount_per_claim):
        square_sum += np.square(amount-paid_mean)
        #print(square_sum)
    df_claim_coverage_each.at[0,'Paid_std'] = np.sqrt(square_sum/accidents_counts)
    
    square_sum = 0
    for amount in (paid_Expenses_per_claim):
        square_sum += np.square(amount-paid_Expenses_Amount_mean)
        #print(square_sum)
    df_claim_coverage_each.at[0,'paid_Expenses_Amount_std'] = np.sqrt(square_sum/accidents_counts)
    
    #df_claim_coverage_each.at[0,'Paid_std'] = df_temp['Paid_Loss_Amount'].std()
    
    #df_claim_coverage_each.at[0,'paid_Expenses_Amount_std'] = df_temp['paid_Expenses_Amount'].std()
    
    
#     for i in df_temp.index:
#         df_claim_coverage_each.at[0,'Policy_Number'] = df_temp.at[i,'Policy_Number']
#         df_claim_coverage_each.at[0,'Accidents_counts'] = df_temp['Claim_Number'].nunique()
#         df_policy_coverage_each.at[0,df_temp.at[i,'Insurance_Coverage']+'_Insured_Amount1'] = df_temp.at[i,'Insured_Amount1']
#         df_policy_coverage_each.at[0,df_temp.at[i,'Insurance_Coverage']+'_Insured_Amount2'] = df_temp.at[i,'Insured_Amount2']
#         df_policy_coverage_each.at[0,df_temp.at[i,'Insurance_Coverage']+'_Insured_Amount3'] = df_temp.at[i,'Insured_Amount3']
#         df_policy_coverage_each.at[0,df_temp.at[i,'Insurance_Coverage']+'_Premium'] = df_temp.at[i,'Premium']
#         total_Premium = total_Premium + df_temp.at[i,'Premium']
#         df_policy_coverage_each['total_Premium'] = total_Premium
        #print(df_policy_coverage_each)
    appended_df_claim_coverage.append(df_claim_coverage_each)
            
    if j % 1000 == 0:
        t2 = time.time()
        print(" {}s (total: {}s)".format((t2-t1)*1000, t2-t0))
    j+=1
appended_df_claim_coverage
df_claim_coverage = pd.concat(appended_df_claim_coverage)

35895
Processing file 1000 (2%) 22.478580474853516s (total: 20.54356575012207s)
Processing file 2000 (5%) 16.565799713134766s (total: 40.27405118942261s)
Processing file 3000 (8%) 20.49851417541504s (total: 60.172683000564575s)
Processing file 4000 (11%) 17.542600631713867s (total: 79.85589957237244s)
Processing file 5000 (13%) 17.595291137695312s (total: 99.65124201774597s)
Processing file 6000 (16%) 17.540693283081055s (total: 119.02261352539062s)
Processing file 7000 (19%) 17.573118209838867s (total: 138.45542430877686s)
Processing file 8000 (22%) 17.566204071044922s (total: 158.46805214881897s)
Processing file 9000 (25%) 24.394750595092773s (total: 178.15150713920593s)
Processing file 10000 (27%) 21.45242691040039s (total: 197.41143679618835s)
Processing file 11000 (30%) 17.568111419677734s (total: 217.69268202781677s)
Processing file 12000 (33%) 18.544673919677734s (total: 237.6469795703888s)
Processing file 13000 (36%) 21.442651748657227s (total: 257.05428409576416s)
Processing f

In [364]:
df_claim_coverage.head(10)

,00I_Paid_mean,00I_Paid_std,00I_Paid_sum,00I_noPaid_percent,01A_Paid_mean,01A_Paid_std,01A_Paid_sum,01A_noPaid_percent,02K_Paid_mean,02K_Paid_std,02K_Paid_sum,02K_noPaid_percent,03L_Paid_mean,03L_Paid_std,03L_Paid_sum,03L_noPaid_percent,04M_Paid_mean,04M_Paid_std,04M_Paid_sum,04M_noPaid_percent,05E_Paid_mean,05E_Paid_std,05E_Paid_sum,05E_noPaid_percent,05N_Paid_mean,05N_Paid_std,05N_Paid_sum,05N_noPaid_percent,06F_Paid_mean,06F_Paid_std,06F_Paid_sum,06F_noPaid_percent,07P_Paid_mean,07P_Paid_std,07P_Paid_sum,07P_noPaid_percent,08H_Paid_mean,08H_Paid_std,08H_Paid_sum,08H_noPaid_percent,09@_Paid_mean,09@_Paid_std,09@_Paid_sum,09@_noPaid_percent,10A_Paid_mean,10A_Paid_std,10A_Paid_sum,10A_noPaid_percent,12L_Paid_mean,12L_Paid_std,12L_Paid_sum,12L_noPaid_percent,14E_Paid_mean,14E_Paid_std,14E_Paid_sum,14E_noPaid_percent,14N_Paid_mean,14N_Paid_std,14N_Paid_sum,14N_noPaid_percent,15F_Paid_mean,15F_Paid_std,15F_Paid_sum,15F_noPaid_percent,15O_Paid_mean,15O_Paid_std,15O_Paid_sum,15O_noPaid_percent,16G_Paid_mean,16G_Paid_std,16G_Paid_sum,16G_noPaid_percent,16P_Paid_mean,16P_Paid_std,16P_Paid_sum,16P_noPaid_percent,18@_Paid_mean,18@_Paid_std,18@_Paid_sum,18@_noPaid_percent,18I_Paid_mean,18I_Paid_std,18I_Paid_sum,18I_noPaid_percent,20B_Paid_mean,20B_Paid_std,20B_Paid_sum,20B_noPaid_percent,20K_Paid_mean,20K_Paid_std,20K_Paid_sum,20K_noPaid_percent,25G_Paid_mean,25G_Paid_std,25G_Paid_sum,25G_noPaid_percent,26H_Paid_mean,26H_Paid_std,26H_Paid_sum,26H_noPaid_percent,29B_Paid_mean,29B_Paid_std,29B_Paid_sum,29B_noPaid_percent,29K_Paid_mean,29K_Paid_std,29K_Paid_sum,29K_noPaid_percent,32N_Paid_mean,32N_Paid_std,32N_Paid_sum,32N_noPaid_percent,33F_Paid_mean,33F_Paid_std,33F_Paid_sum,33F_noPaid_percent,33O_Paid_mean,33O_Paid_std,33O_Paid_sum,33O_noPaid_percent,34P_Paid_mean,34P_Paid_std,34P_Paid_sum,34P_noPaid_percent,35H_Paid_mean,35H_Paid_std,35H_Paid_sum,35H_noPaid_percent,36I_Paid_mean,36I_Paid_std,36I_Paid_sum,36I_noPaid_percent,40M_Paid_mean,40M_Paid_std,40M_Paid_sum,40M_noPaid_percent,41E_Paid_mean,41E_Paid_std,41E_Paid_sum,41E_noPaid_percent,41N_Paid_mean,41N_Paid_std,41N_Paid_sum,41N_noPaid_percent,42F_Paid_mean,42F_Paid_std,42F_Paid_sum,42F_noPaid_percent,45@_Paid_mean,45@_Paid_std,45@_Paid_sum,45@_noPaid_percent,46A_Paid_mean,46A_Paid_std,46A_Paid_sum,46A_noPaid_percent,51O_Paid_mean,51O_Paid_std,51O_Paid_sum,51O_noPaid_percent,55J_Paid_mean,55J_Paid_std,55J_Paid_sum,55J_noPaid_percent,56B_Paid_mean,56B_Paid_std,56B_Paid_sum,56B_noPaid_percent,56K_Paid_mean,56K_Paid_std,56K_Paid_sum,56K_noPaid_percent,57C_Paid_mean,57C_Paid_std,57C_Paid_sum,57C_noPaid_percent,57L_Paid_mean,57L_Paid_std,57L_Paid_sum,57L_noPaid_percent,66L_Paid_mean,66L_Paid_std,66L_Paid_sum,66L_noPaid_percent,70G_Paid_mean,70G_Paid_std,70G_Paid_sum,70G_noPaid_percent,70P_Paid_mean,70P_Paid_std,70P_Paid_sum,70P_noPaid_percent,Accidents_counts,At_Fault_mean,Paid_mean,Paid_std,Paid_sum,Policy_Number,noPaid_percent,paid_Expenses_Amount_mean,paid_Expenses_Amount_std,paid_Expenses_Amount_sum
0,28352.5,40096.490027,56705.0,0.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1100.0,NaN,1100.0,0.0,15000.000000,21213.203436,30000.0,0.500000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,50.0,43902.50,43902.500000,87805.0,91dc13e4e24dc9e0e0940c63924109e4ff66e18a,0.5,850.0,150.000000,1700.0
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

In [365]:
df_claim_coverage.to_csv('../Tbrain_Insurance/df_claim_coverage.csv', index = False)

In [189]:
df_claim.loc[(df_claim['Policy_Number']=='79edd5355bf511649ffefe614bbd6e46d4a00474')]

,Claim_Number,Nature_of_the_claim,Policy_Number,Driver's_Gender,Driver's_Relationship_with_Insured,DOB_of_Driver,Marital_Status_of_Driver,Accident_Date,Cause_of_Loss,Paid_Loss_Amount,paid_Expenses_Amount,Salvage_or_Subrogation?,Coverage,Vehicle_identifier,At_Fault?,"Claim_Status_(close,_open,_reopen_etc)",Deductible,Accident_area,number_of_claimants,Accident_Time
43332,daee7a8400e32ec7ba02286aed3464b28425df98,1,79edd5355bf511649ffefe614bbd6e46d4a00474,1,1,03/1983,1,2015/10,bd307a3ec329e10a2cff8fb87480823da114f8f4,0,0,0,16P,fd0a1696e8332e414f8fc47ef769e66d30489663,0.0,1,0,98fbc42faedc02492397cb5962ea3a3ffc0a9243,1.787,12:00
43333,daee7a8400e32ec7ba02286aed3464b28425df98,1,79edd5355bf511649ffefe614bbd6e46d4a00474,1,1,03/1983,1,2015/10,bd307a3ec329e10a2cff8fb87480823da114f8f4,261481,0,0,04M,fd0a1696e8332e414f8fc47ef769e66d30489663,0.0,1,0,98fbc42faedc02492397cb5962ea3a3ffc0a9243,1.787,12:00
43334,daee7a8400e32ec7ba02286aed3464b28425df98,2,79edd5355bf511649ffefe614bbd6e46d4a00474,1,1,03/1983,1,2015/10,bd307a3ec329e10a2cff8fb87480823da114f8f4,0,0,258331,04M,fd0a1696e8332e414f8fc47ef769e66d30489663,0.0,1,0,98fbc42faedc02492397cb5962ea3a3ffc0a9243,1.787,12:00
43335,daee7a8400e32ec7ba02286aed3464b28425df98,2,79edd5355bf511649ffefe614bbd6e46d4a00474,1,1,03/1983,1,2015/10,bd307a3ec329e10a2cff8fb87480823da114f8f4,0,0,0,16P,fd0a1696e8332e414f8fc47ef769e66d30489663,0.0,1,0,98fbc42faedc02492397cb5962ea3a3ffc0a9243,1.787,12:00
51038,e7a78725a3d4146e0cdc708216b09534e65934cb,1,79edd5355bf511649ffefe614bbd6e46d4a00474,1,1,03/1983,1,2016/07,bd307a3ec329e10a2cff8fb87480823da114f8f4,0,0,0,04M,fd0a1696e8332e414f8fc47ef769e66d30489663,0.0,1,0,761f22b2c1593d0bb87e0b606f990ba4974706de,1.787,21:00
51039,e7a78725a3d4146e0cdc708216b09534e65934cb,1,79edd5355bf511649ffefe614bbd6e46d4a00474,1,1,03/1983,1,2016/07,bd307a3ec329e10a2cff8fb87480823da114f8f4,0,0,0,16P,fd0a1696e8332e414f8fc47ef769e66d30489663,0.0,1,0,761f22b2c1593d0bb87e0b606f990ba4974706de,1.787,21:00
51040,e7a78725a3d4146e0cdc708216b09534e65934cb,1,79edd5355bf511649ffefe614bbd6e46d4a00474,1,1,03/1983,1,2016/07,bd307a3ec329e10a2cff8fb87480823da114f8f4,0,0,0,16G,fd0a1696e8332e414f8fc47ef769e66d30489663,0.0,1,0,761f22b2c1593d0bb87e0b606f990ba4974706de,1.787,21:00
51113,dbb07df8eada693578e65162f5dcf7dbf3633483,1,79edd5355bf511649ffefe614bbd6e46d4a00474,1,1,03/1983,1,2016/07,bd307a3ec329e10a2cff8fb87480823da114f8f4,26835,0,0,04M,fd0a1696e8332e414f8fc47ef769e66d30489663,0.0,1,0,761f22b2c1593d0bb87e0b606f990ba4974706de,1.787,21:00
51114,dbb07df8eada693578e65162f5dcf7dbf3633483,1,79edd5355bf511649ffefe614bbd6e46d4a00474,1,1,03/1983,1,2016/07,bd307a3ec329e10a2cff8fb87480823da114f8f4,0,0,0,16P,fd0a1696e8332e414f8fc47ef769e66d30489663,0.0,1,0,761f22b2c1593d0bb87e0b606f990ba4974706de,1.787,21:00
51115,dbb07df8eada693578e65162f5dcf7dbf3633483,1,79edd5355bf511649ffefe614bbd6e46d4a00474,1,1,03/1983,1,2016/07,bd307a3ec329e10a2cff8fb87480823da114f8f4,0,0,0,16G,fd0a1696e8332e414f8fc47ef769e66d30489663,0.0,1,0,761f22b2c1593d0bb87e0b606f990ba4974706de,1.787,21:00


# 8069個保單中，一個險種對應多個

In [95]:
count = df_claim.groupby("Policy_Number")['Coverage'].nunique()
count
count2 = df_claim.groupby("Policy_Number")['Coverage'].count()
count2.index

j=0
list=[]
for i in count.index:
    if count[i] < count2[i]:
        list.append(i)
        j=j+1
print(list[0:5])
print(str(j)+'個保單中，一個險種對應多個')

['0004a3f30deef9fc8f9f5cb334b8bec8d79c7453', '000949143e8dc7d5f570cb8c3b6aa51c8f8ac9ad', '0009dfd25805f44c7fa82c1ddb010eb3a05612f6', '000a5d472fa501c2e9d771dfae4af646f281f97d', '000e39e8a273654b6fc4132ba81b28e34d9463f2']
8069個保單中，一個險種對應多個


# 確認險種在policy中，每個險種為unique

In [368]:
count = df_policy.groupby("Policy_Number")['Insurance_Coverage'].nunique()
count
count2 = df_policy.groupby("Policy_Number")['Insurance_Coverage'].count()
count2

Policy_Number
000061141b237e8619efedcd6939fddeff05b9a5     3
0000d65ef9b4e9e5e9603617bd5fb8497b7f02b5     7
0001221abee6f59d6d40eeb1ba9ffa0f4f92b36b     3
000148bbf40f28bfc860c0d272aadc429e19e2d1     2
0001ad0ad0618f6d2e2c1875105adce6bcd77c2c     3
0001e3ed1dc0e7daf8316d54c07dbafcea01120c     4
00022176b92ea50700fea7e2003cd25cfd2ccc5c     2
00026ca0e304a4be19f0cec1121ff33ef81146c5     7
000278a1dbf89c761eaeb76c9a3a1cf9d74f2cf0     7
000315d06d5dfe88d1cfa138e4dc77e650f6d674     6
00036f9419b02dff17b3b49b7f27b3c38a29913a     6
0003a6128508290abb780d735fac98b2440d162e     5
0003c9e6c6d92dda5f000e57b1e473b2547daea0     9
0003da9a3dac9c40813cf77222a271959d77e73d     5
0003e5d7127edb8fe1eea972d2eb2ff616e4c059     6
0003ff2ff52a8214b898fa8f8dd2f81637d21192     5
00042af28f34e60c9ec45d681078a1c30dfe1c2e    10
00042bac2d5e4eabdee93aa9933c4cea3318069e     4
0004a1f247d6aa00e195dfab2aa3825d321b0409     2
0004a3f30deef9fc8f9f5cb334b8bec8d79c7453    12
0005264e35be6bc039a87b8c63330cc71389b003     2

In [369]:
for i in count.index:
    if count[i] != count2[i]:
        print(i)
print('done')

done


# ---------------------------------------------------------------------------